# 朴素贝叶斯分类

In [1]:
import re
import numpy as np
import pandas as pd

In [2]:
#加载数据
spam = []
ham = []
for i in range(1, 26):
    spam.append(open('spam/%d.txt' % i, 'r').read())
    ham.append(open('ham/%d.txt' % i, 'r').read())

In [3]:
for i in range(len(ham)):
    ham[i] = re.sub('\W', ' ', ham[i].lower())
    ham[i] = re.sub('\d', ' ', ham[i].lower())
for i in range(len(ham)):
    spam[i] = re.sub('\W', ' ', spam[i].lower())
    spam[i] = re.sub('\d', ' ', spam[i].lower())

In [56]:
class Bayes:
    def __init__(self):     
        #单词总数
        self.ham_words_count = 0
        self.spam_words_count = 0

        #所含的所有单词
        self.ham_words = []
        self.spam_words = []

        #每种短信的数量
        self.ham_count = 0
        self.spam_count = 0

        #每种短信中各单词出现对的次数
        self.ham_map = {}
        self.spam_map = {}
        
    def data_deal(self, ham, spam):
        self.ham_count = len(ham)
        self.spam_count = len(spam)
        
        for i in range(len(ham)):
            temp_1 = re.split(r'\s+', ham[i])
            for j in range(len(temp_1)):
                if temp_1[j] != '':
                    self.ham_words.append(temp_1[j])
                    self.ham_words_count += 1

        for i in range(len(spam)):
            temp_2 = re.split(r'\s+', spam[i])
            for j in range(len(temp_2)):
                if temp_2[j] != '':
                    self.spam_words.append(temp_2[j])
                    self.spam_words_count += 1

        self.word_dictionary_set = set(self.ham_words) | set(self.spam_words)

        for word in self.ham_words:
            if word not in self.ham_map.keys():
                self.ham_map[word] = 0
            self.ham_map[word] += 1
        for word in self.spam_words:
            if word not in self.spam_map.keys():
                self.spam_map[word] = 0
            self.spam_map[word] += 1  
        return self
    
    def predict(self, message):
        try:
            prob_1 = 0.0
            prob_2 = 0.0
            P_1 = (self.ham_count + 1) / (self.spam_count + self.ham_count + 2)
            P_2 = (self.spam_count + 1) / (self.spam_count + self.ham_count + 2)
            message_dict = {}
            message = re.sub('\W', ' ', message.lower())
            message = re.sub('\d', ' ', message.lower())
            message = re.split(r'\s+', message)
            for word in message:
                if word not in message_dict:
                    message_dict[word] = 0
                message_dict[word] += 1
            prob = 1.0
            for word in message_dict.keys():
                if word not in self.ham_map.keys():
                    p = 1 / (self.ham_count + 1)
                else:
                    p = (1 + message_dict[word]) / (self.ham_map[word] + 1)
                prob *= p
            prob_1 = prob*P_1
            prob = 1.0
            for word in message_dict.keys():
                if word not in self.spam_map.keys():
                    p = 1 / (self.spam_count + 1)
                else:
                    p = (1 + message_dict[word]) / (self.spam_map[word] + 1)
                prob *= p
            prob_2 = prob*P_2
            if prob_1 > prob_2:
                return ('ham')
            else:
                return('spam')
        except Exception as e:
            print(e)